In [30]:
!pip install python-docx pypdf faiss-cpu transformers torch tqdm sentence-transformers fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.7 MB/s eta 0:00:00


In [40]:
import os
from typing import List, Dict, Union
from docx import Document
from pypdf import PdfReader
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel
import torch
import pickle
from google.colab import drive, userdata
from pathlib import Path
import json
from datetime import datetime
import uuid
import logging
from tqdm import tqdm
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from pydantic import BaseModel
from fastapi import FastAPI, HTTPException

nest_asyncio.apply()
def setup_google_drive():
    """Mount Google Drive and return the base path"""
    try:
        drive.mount('/content/drive')
        base_path = '/content/drive/My Drive/lifesciences'
        print(f"Google Drive mounted, base path: {base_path}")
        return base_path
    except Exception as e:
        logging.error(f"Error mounting Google Drive: {e}", exc_info=True)
        print("Error mounting Google Drive.")
        return './lifesciences'  # Default to local directory if Google Drive fails

class DocumentProcessor:
    def __init__(self, base_path: str, model_name: str = "sentence-transformers/all-mpnet-base-v2"):
        self.base_path = base_path
        print(f"Base path: {self.base_path}")  # Consider using logging instead of print
        self.models_path = os.path.join(base_path, 'models')
        print(f"Models path: {self.models_path}")  # Consider using logging instead of print
        self.index_path = os.path.join(base_path, 'indexes')
        print(f"Index path: {self.index_path}")  # Consider using logging instead of print
        self.extracted_data_path = os.path.join(base_path, 'extracted_data')
        print(f"Extracted data path: {self.extracted_data_path}")  # Consider using logging instead of print
        self.embeddings_path = os.path.join(base_path, 'embeddings')
        print(f"Embeddings path: {self.embeddings_path}")  # Consider using logging instead of print
        self.training_docs_path = os.path.join(base_path, 'training_documents')
        print(f"Training docs path: {self.training_docs_path}")  # Consider using logging instead of print
        self.chunks = []
        # Create directories if they don't exist
        for path in [self.models_path, self.index_path,
                    self.extracted_data_path, self.embeddings_path,
                    self.training_docs_path]:
            os.makedirs(path, exist_ok=True)
            print(f"Ensured directory exists: {path}")

        # Setup logging
        self.setup_logging()

        # Initialize the model
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            print(f"Tokenizer loaded for model {model_name}")
            self.model = AutoModel.from_pretrained(model_name)
            print(f"Model loaded for {model_name}")
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(f"Device set to: {self.device}")
            self.model.to(self.device)
            self.logger.info(f"Model '{model_name}' loaded successfully on {self.device}.")  # Log success
        except Exception as e:
            self.logger.error(f"Error initializing model: {e}", exc_info=True)
            print(f"Error initializing model: {e}")
            raise  # Re-raise to stop execution if model fails to load

    def setup_logging(self):
        logging.basicConfig(
            filename=os.path.join(self.base_path, 'processing.log'),
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)
        self.logger.info("Logging initialized.")  # Confirm logging setup
        print("Logging initialized.")

    def get_document_metadata(self, file_path: str) -> Dict:
        """Extract metadata from document"""
        try:
            file_stat = os.stat(file_path)
            metadata = {
                "filename": os.path.basename(file_path),
                "file_path": file_path,
                "file_type": file_path.split('.')[-1].lower(),
                "size": file_stat.st_size,
                "created": datetime.fromtimestamp(file_stat.st_ctime).isoformat(),
                "modified": datetime.fromtimestamp(file_stat.st_mtime).isoformat(),
                "processed": datetime.now().isoformat()
            }
            print(f"Metadata extracted for {file_path}")
            self.logger.info(f"Metadata extracted for {file_path}")
            return metadata
        except Exception as e:
            self.logger.error(f"Error getting metadata for {file_path}: {e}", exc_info=True)
            print(f"Error getting metadata for {file_path}: {e}")
            return {}  # Return an empty dict in case of an error

    def extract_and_save_content(self, file_path: str) -> str:
        """Extract content from a document and save it as JSON"""
        try:
            metadata = self.get_document_metadata(file_path)
            extracted_data = []

            # Read document content
            if file_path.endswith('.docx'):
                extracted_data = self.extract_text_from_docx(file_path)
            elif file_path.endswith('.pdf'):
                extracted_data = self.extract_text_from_pdf(file_path)
            else:
                self.logger.warning(f"Unsupported file type: {file_path}")
                print(f"Unsupported file type: {file_path}")
                return None  # Handle unsupported types

            # Save extracted content and metadata
            extracted_json = {
                "metadata": metadata,
                "sections": extracted_data
            }

            extracted_file_path = os.path.join(self.extracted_data_path, f"{uuid.uuid4()}.json")
            with open(extracted_file_path, 'w', encoding='utf-8') as f:
                json.dump(extracted_json, f, ensure_ascii=False, indent=4)

            self.logger.info(f"Content extracted and saved to {extracted_file_path}")
            print(f"Content extracted and saved to {extracted_file_path}")
            return extracted_file_path
        except Exception as e:
            self.logger.error(f"Error extracting and saving content from {file_path}: {e}", exc_info=True)
            print(f"Error extracting and saving content from {file_path}: {e}")
            return None

    def extract_text_from_docx(self, file_path: str) -> List[Dict[str, str]]:
        """Extract text from a Word document (.docx)"""
        try:
            doc = Document(file_path)
            sections = []

            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    sections.append({"text": paragraph.text.strip()})
            print(f"Text extracted from DOCX: {file_path}")
            self.logger.info(f"Text extracted from DOCX: {file_path}")
            return sections
        except Exception as e:
            self.logger.error(f"Error extracting text from DOCX: {e}", exc_info=True)
            print(f"Error extracting text from DOCX: {e}")
            return []  # Return empty list in case of error

    def extract_text_from_pdf(self, file_path: str) -> List[Dict[str, str]]:
        """Extract text from a PDF document"""
        try:
            reader = PdfReader(file_path)
            sections = []

            for page in reader.pages:
                text = page.extract_text()
                if text.strip():
                    sections.append({"text": text.strip()})
            print(f"Text extracted from PDF: {file_path}")
            self.logger.info(f"Text extracted from PDF: {file_path}")
            return sections
        except Exception as e:
            self.logger.error(f"Error extracting text from PDF: {e}", exc_info=True)
            print(f"Error extracting text from PDF: {e}")
            return []  # Return empty list in case of error

    def get_embeddings(self, text: str) -> np.ndarray:
        """Generate sentence embeddings for a given text"""
        try:
            inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True)
            inputs = inputs.to(self.device)
            print(f"Tokenizing input: {text}")

            with torch.no_grad():
                output = self.model(**inputs)
                print(f"Model output generated for input text.")

            # The error was likely happening because the model returns more than just the embedding.
            # We specifically want the last hidden state and mean it to get a sentence embedding.
            embedding = output.last_hidden_state.mean(dim=1).cpu().numpy()
            print(f"Generated embedding for text.")
            return embedding
        except Exception as e:
            self.logger.error(f"Error generating embeddings for text: {e}", exc_info=True)
            print(f"Error generating embeddings for text: {e}")
            return None

    def process_and_save_embeddings(self, sections: List[Dict[str, str]]) -> str:
        """Process sections and save their embeddings"""
        try:
            embeddings = []
            for section in sections:
                text = section["text"]
                embedding = self.get_embeddings(text)
                if embedding is not None:
                    section["embedding"] = embedding.tolist()  # Convert to list for JSON serialization
                    embeddings.append(section)
                else:
                    self.logger.warning(f"Could not generate embedding for section: {section}")
                    print(f"Could not generate embedding for section: {section}")

            # Save embeddings to file
            embedding_file_path = os.path.join(self.embeddings_path, f"{uuid.uuid4()}.pkl")
            with open(embedding_file_path, 'wb') as f:
                pickle.dump(embeddings, f)

            # Ensure chunks are updated
            self.chunks.extend(embeddings)
            self.logger.info(f"Embeddings processed and saved to {embedding_file_path}")
            print(f"Embeddings processed and saved to {embedding_file_path}")
            print(f"Current number of chunks: {len(self.chunks)}")  # Debugging line
            return embedding_file_path
        except Exception as e:
            self.logger.error(f"Error processing and saving embeddings: {e}", exc_info=True)
            print(f"Error processing and saving embeddings: {e}")
            return None  # Indicate failure



    def process_training_documents(self):
        training_docs_path = self.training_docs_path
        extracted_data_path = self.extracted_data_path
        logger = self.logger
        """Processes all documents in the training documents folder."""
        processed_files = []
        try:
            for filename in os.listdir(training_docs_path):
                if filename.endswith(('.docx', '.pdf')):
                    file_path = os.path.join(training_docs_path, filename)
                    extracted_file_path = self.extract_and_save_content(file_path)
                    if extracted_file_path:
                        processed_files.append(extracted_file_path)
            logger.info(f"Processed {len(processed_files)} training documents.")
            print(f"Processed {len(processed_files)} training documents.")
            return processed_files
        except Exception as e:
            logger.error(f"Error processing training documents: {e}", exc_info=True)
            print(f"Error processing training documents: {e}")
            return processed_files

class DocumentSearch:
    def __init__(self, base_path: str, dimension: int = 768):
        self.base_path = base_path
        self.index_path = os.path.join(base_path, 'indexes')
        self.embeddings_path = os.path.join(base_path, 'embeddings')
        self.processor = DocumentProcessor(base_path)
        print(f"DocumentSearch initialized with base path {base_path}")

        # Initialize FAISS index
        self.dimension = dimension
        self.index = faiss.IndexFlatL2(dimension)
        self.chunks = []  # Ensure this list is populated with sections

        # Load existing index if available
        self.load_index()

    def load_chunks(self):
        """Load chunks from a previously saved file"""
        chunks_file = os.path.join(self.index_path, 'chunks.pkl')
        if os.path.exists(chunks_file):
            try:
                with open(chunks_file, 'rb') as f:
                    self.chunks = pickle.load(f)
                self.processor.logger.info(f"Loaded {len(self.chunks)} chunks.")
                print(f"Loaded {len(self.chunks)} chunks.")
            except Exception as e:
                self.processor.logger.error(f"Error loading chunks: {e}", exc_info=True)
                print(f"Error loading chunks: {e}")
                self.chunks = []  # Initialize as empty list if loading fails
        else:
            self.processor.logger.info("No chunks file found, starting fresh.")
            print("No chunks file found, starting fresh.")
            self.chunks = []  # Initialize as empty list if file doesn't exist

    def save_chunks(self):
        """Save chunks to a file for future use"""
        try:
            chunks_file = os.path.join(self.index_path, 'chunks.pkl')
            with open(chunks_file, 'wb') as f:
                pickle.dump(self.chunks, f)
            self.processor.logger.info(f"Saved {len(self.chunks)} chunks to file.")
            print(f"Saved {len(self.chunks)} chunks to file.")
        except Exception as e:
            self.processor.logger.error(f"Error saving chunks: {e}", exc_info=True)
            print(f"Error saving chunks: {e}")

    def load_index(self):
        """Load the FAISS index from file if it exists"""
        index_file = os.path.join(self.index_path, 'index.faiss')
        if os.path.exists(index_file):
            try:
                self.index = faiss.read_index(index_file)
                print(f"Loaded index with {self.index.ntotal} vectors.")  # Debugging line
                self.load_chunks()  # Ensure chunks are loaded after the index is loaded
                self.processor.logger.info(f"Index loaded with {self.index.ntotal} vectors.")
            except Exception as e:
                self.processor.logger.error(f"Error loading index: {str(e)}")
                print(f"Error loading index: {str(e)}")
                self.index = faiss.IndexFlatL2(self.dimension)  # Initialize a new index if loading fails
                self.chunks = []  # Initialize chunks as empty if index loading fails
        else:
            self.processor.logger.info("No index found, starting fresh.")
            print("No index found, starting fresh.")
            self.index = faiss.IndexFlatL2(self.dimension)  # Initialize a new index if none exists
            self.chunks = []  # Initialize as empty list if no index file


    def process_training_documents(self):
        """Process documents, add embeddings to index, and store chunks"""
        processed_files = self.processor.process_training_documents()

        # Generate embeddings and store chunks
        all_embeddings = []
        for file_path in processed_files:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                sections = data.get("sections", [])
                embeddings_file_path = self.processor.process_and_save_embeddings(sections)

                # Load embeddings from the file
                if embeddings_file_path:
                    with open(embeddings_file_path, 'rb') as f:
                        embeddings = pickle.load(f)
                        for section in embeddings:
                            if "embedding" in section:
                                embedding = np.array(section["embedding"], dtype=np.float32)
                                all_embeddings.append(embedding)
                                self.chunks.append(section)  # Save section with embedding
        # Add embeddings to FAISS index
        if all_embeddings:
            embeddings_matrix = np.vstack(all_embeddings)
            self.index.add(embeddings_matrix)
            self.processor.logger.info(f"Added {len(all_embeddings)} embeddings to the FAISS index.")
            print(f"Added {len(all_embeddings)} embeddings to the FAISS index.")
            self.save_chunks()  # Save the chunks to file





    def query(self, query_text: str, top_k: int = 5) -> List[Dict[str, Union[str, float]]]:
        """Query the FAISS index"""
        try:
            query_embedding = self.processor.get_embeddings(query_text)
            if query_embedding is not None:
                query_embedding = query_embedding.astype(np.float32)
                distances, indices = self.index.search(query_embedding, top_k)

                # Ensure indices are valid and match the chunks
                if len(indices[0]) > len(self.chunks):
                    self.processor.logger.warning(f"Indices returned exceed number of chunks. Check index and chunks integrity.")
                    print(f"Warning: Indices returned exceed number of chunks. Check index and chunks integrity.")
                    return []

                results = []
                for idx, dist in zip(indices[0], distances[0]):
                    if idx < len(self.chunks):
                        result = {
                            "text": self.chunks[idx]["text"],
                            "distance": dist
                        }
                        results.append(result)

                self.processor.logger.info(f"Query results: {results}")
                print(f"Query results: {results}")
                combined_results = "\n".join([r['text'] for r in results])
                response = [{"generated_text": combined_results}]
                return response

        except Exception as e:
              print(f"❌ Error handling query: {e}")
              response = [{"generated_text": "Sorry, no training data available for this query"}]
              return response

# Define the FastAPI app
app = FastAPI()

class QueryInput(BaseModel):
    inputs: str
# FastAPI Endpoints
@app.on_event("startup")
async def startup_event():
    ngrok.set_auth_token(userdata.get('ngrok_auth_token'))  # Replace with actual token if needed
    public_url = ngrok.connect(8000)
    print(f"API available at: {public_url}")

@app.post("/query")
async def query_section(query: QueryInput):
    # Example query
    print(f"📥 Received query: {query.inputs}")
    query_text = query.inputs
    base_path = '/content/drive/My Drive/lifesciences'
    document_search = DocumentSearch(base_path)
    response = document_search.query(query_text, 10)
    return response

def main():

    base_path = setup_google_drive()
    document_processor = DocumentProcessor(base_path)
    document_search = DocumentSearch(base_path)
    # Process training documents
    document_search.process_training_documents()
    uvicorn.run(app, host="0.0.0.0", port=8000)



if __name__ == "__main__":
    main()


<ipython-input-40-ac9f35b825b5>:390: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted, base path: /content/drive/My Drive/lifesciences
Base path: /content/drive/My Drive/lifesciences
Models path: /content/drive/My Drive/lifesciences/models
Index path: /content/drive/My Drive/lifesciences/indexes
Extracted data path: /content/drive/My Drive/lifesciences/extracted_data
Embeddings path: /content/drive/My Drive/lifesciences/embeddings
Training docs path: /content/drive/My Drive/lifesciences/training_documents
Ensured directory exists: /content/drive/My Drive/lifesciences/models
Ensured directory exists: /content/drive/My Drive/lifesciences/indexes
Ensured directory exists: /content/drive/My Drive/lifesciences/extracted_data
Ensured directory exists: /content/drive/My Drive/lifesciences/embeddings
Ensured directory exists: /content/drive/My Drive/lifesciences/training_documents
Logging initialized.
Tokenizer loaded for model se

INFO:     Started server process [33686]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


API available at: NgrokTunnel: "https://a997-34-23-7-24.ngrok-free.app" -> "http://localhost:8000"
📥 Received query: Tell me some points on machine safety
Base path: /content/drive/My Drive/lifesciences
Models path: /content/drive/My Drive/lifesciences/models
Index path: /content/drive/My Drive/lifesciences/indexes
Extracted data path: /content/drive/My Drive/lifesciences/extracted_data
Embeddings path: /content/drive/My Drive/lifesciences/embeddings
Training docs path: /content/drive/My Drive/lifesciences/training_documents
Ensured directory exists: /content/drive/My Drive/lifesciences/models
Ensured directory exists: /content/drive/My Drive/lifesciences/indexes
Ensured directory exists: /content/drive/My Drive/lifesciences/extracted_data
Ensured directory exists: /content/drive/My Drive/lifesciences/embeddings
Ensured directory exists: /content/drive/My Drive/lifesciences/training_documents
Logging initialized.
Tokenizer loaded for model sentence-transformers/all-mpnet-base-v2
Model 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [33686]
